# Assignment 4: Pipelines and Text Data (60 total marks)
### Due: March 21 at 11:59pm

### Name: 

In [8]:
import numpy as np
import pandas as pd

In [9]:
import warnings
warnings.filterwarnings('ignore') #ignoring some deprication warnings

## Part 1: Pipelines (26 marks)

The purpose of this part of the assignment is to practice following the grid-search workflow: 
- Split data into training and test set
- Use the training portion to find the best model using grid search and cross-validation
- Retrain the best model
- Evaluate the retrained model on the test set

### 1.1: Load data (4 marks)
For this task, we will be using stock data from the Dow Jones Index. This dataset uses information about different stocks to try to predict what the percent change in price will be from week to week.

More information on the dataset can be found here: https://archive.ics.uci.edu/dataset/312/dow+jones+index

In [10]:
# TO DO: Load the dataset into a dataframe called stock_data (0.5 marks)
stock_data = pd.read_csv('dow_jones_index.data')

# TO DO: Inspect the first few columns (0.5 marks)
print(stock_data.head())


   quarter stock       date    open    high     low   close     volume  \
0        1    AA   1/7/2011  $15.82  $16.72  $15.78  $16.42  239655616   
1        1    AA  1/14/2011  $16.71  $16.71  $15.64  $15.97  242963398   
2        1    AA  1/21/2011  $16.19  $16.38  $15.60  $15.79  138428495   
3        1    AA  1/28/2011  $15.87  $16.63  $15.82  $16.13  151379173   
4        1    AA   2/4/2011  $16.18  $17.39  $16.18  $17.14  154387761   

   percent_change_price  percent_change_volume_over_last_wk  \
0               3.79267                                 NaN   
1              -4.42849                            1.380223   
2              -2.47066                          -43.024959   
3               1.63831                            9.355500   
4               5.93325                            1.987452   

   previous_weeks_volume next_weeks_open next_weeks_close  \
0                    NaN          $16.71           $15.97   
1            239655616.0          $16.19           $15

In [11]:
# TO DO: Check the data types of each column and if there are missing values (0.5 marks)
print(stock_data.info())
print("-----------------------------------------------------------------------")
print("\nMissing values:\n", stock_data.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   quarter                             750 non-null    int64  
 1   stock                               750 non-null    object 
 2   date                                750 non-null    object 
 3   open                                750 non-null    object 
 4   high                                750 non-null    object 
 5   low                                 750 non-null    object 
 6   close                               750 non-null    object 
 7   volume                              750 non-null    int64  
 8   percent_change_price                750 non-null    float64
 9   percent_change_volume_over_last_wk  720 non-null    float64
 10  previous_weeks_volume               720 non-null    float64
 11  next_weeks_open                     750 non-n

You should notice in this dataset that there are multiple columns that look numerical, but include a `$` that turns the value into a string (type object). You can use the code below to convert these columns into numerical ones:

In [12]:
# TO DO: Fill-in which columns need the $ to be removed (1 mark)
columns = ['open', 'high', 'low', 'close', 'next_weeks_open', 'next_weeks_close']

# Code to remove $ - DO NOT CHANGE
stock_data[columns] = stock_data[columns].replace('[\$]', '', regex=True).astype(float)

# TO DO: Inspect first few rows to make sure it worked (0.5 marks)
print(stock_data.head())

   quarter stock       date   open   high    low  close     volume  \
0        1    AA   1/7/2011  15.82  16.72  15.78  16.42  239655616   
1        1    AA  1/14/2011  16.71  16.71  15.64  15.97  242963398   
2        1    AA  1/21/2011  16.19  16.38  15.60  15.79  138428495   
3        1    AA  1/28/2011  15.87  16.63  15.82  16.13  151379173   
4        1    AA   2/4/2011  16.18  17.39  16.18  17.14  154387761   

   percent_change_price  percent_change_volume_over_last_wk  \
0               3.79267                                 NaN   
1              -4.42849                            1.380223   
2              -2.47066                          -43.024959   
3               1.63831                            9.355500   
4               5.93325                            1.987452   

   previous_weeks_volume  next_weeks_open  next_weeks_close  \
0                    NaN            16.71             15.97   
1            239655616.0            16.19             15.79   
2          

In [13]:
# TO DO: Check data type of each column to make sure that the type of the columns selected has changed (0.5 marks)
print(stock_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   quarter                             750 non-null    int64  
 1   stock                               750 non-null    object 
 2   date                                750 non-null    object 
 3   open                                750 non-null    float64
 4   high                                750 non-null    float64
 5   low                                 750 non-null    float64
 6   close                               750 non-null    float64
 7   volume                              750 non-null    int64  
 8   percent_change_price                750 non-null    float64
 9   percent_change_volume_over_last_wk  720 non-null    float64
 10  previous_weeks_volume               720 non-null    float64
 11  next_weeks_open                     750 non-n

The first thing we need to do is deal with missing values. Looking at the dataset, there are two columns with 30 missing values. For this case, we will drop these rows instead of filling them in.

In [14]:
# TO DO: Drop rows with missing data (0.5 marks)
stock_data = stock_data.dropna()

# TO DO: Verify that missing values are removed (0.5 marks)
print("\nMissing values after drop:\n", stock_data.isnull().sum())


Missing values after drop:
 quarter                               0
stock                                 0
date                                  0
open                                  0
high                                  0
low                                   0
close                                 0
volume                                0
percent_change_price                  0
percent_change_volume_over_last_wk    0
previous_weeks_volume                 0
next_weeks_open                       0
next_weeks_close                      0
percent_change_next_weeks_price       0
days_to_next_dividend                 0
percent_return_next_dividend          0
dtype: int64


### 1.2: Pre-processing (4 marks)

In this dataset, we have columns with:
- Categorical values
- Numerical values

We need to create a column transformer that will use the proper preprocessing methods on each type of column.

In [15]:
# TO DO: Create Column Transformer using an encoder and StandardScaler (1 mark)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    [("scaling", StandardScaler(), ['quarter', 'open', 'high', 'low', 'close', 'volume', 'next_weeks_open', 'next_weeks_close']),
     ("onehot", OneHotEncoder(sparse_output=False), ['stock'])])

In [16]:
# TO DO: Initialize your pipeline with your column transformer and the Ridge Regression model (1 mark)
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', Ridge())])

In [17]:
# TO DO: Separate data into feature matrix and target vector (1 mark)
# On the website, the column percent_change_next_weeks_price is the target
X = stock_data.drop(columns='percent_change_next_weeks_price')
y = stock_data['percent_change_next_weeks_price']

In [18]:
# TO DO: Split data into training and testing sets (use random_state=0 and 10% of the data for testing) (0.5 marks)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

Create another column transformer that does not implement scaling

In [19]:
# TO DO: Create a new column transformer that only performs encoding (0.5 marks)
preprocessor_encoding_only = ColumnTransformer(
    [("onehot", OneHotEncoder(sparse_output=False), ['stock'])]
)


### 1.3: Grid Search (4 marks)

For the grid search, we want to compare the performance of the Random Forest model to a Ridge Regression model with the two different column transformers. Think about if we need to use scaling for both models. Select parameter values to test that make sense for both models.

In [20]:
# TO DO: Create parameter grid and initialize grid object (3 marks)
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

ridge = Ridge()
random_forest = RandomForestRegressor(random_state=42)

# Pipelines
pipeline_ridge = Pipeline([
    ('preprocessor', preprocessor),
    ('model', ridge)
])

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor_encoding_only),
    ('model', random_forest)
])

#Ridge Regression
ridge_params = {
    'model__alpha': [0.01, 0.1, 1, 10, 100]
}

#Random Forest
rf_params = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_split': [2, 5, 10]
}

grid_ridge = GridSearchCV(estimator=pipeline_ridge, param_grid=ridge_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_rf = GridSearchCV(estimator=pipeline_rf, param_grid=rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [21]:
# TO DO: Fit grid object to training data (1 mark)
grid_ridge.fit(X_train, y_train)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('onehot',
                                                                         OneHotEncoder(sparse_output=False),
                                                                         ['stock'])])),
                                       ('model',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [None, 10, 20],
                         'model__min_samples_split': [2, 5, 10],
                         'model__n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

### 1.4: Visualize Results (2 marks)

The final step is to print out the results from the grid search. You will need to print out the following items:
- Best parameters
- Best cross-validation train score 
- Best cross-validation test score
- Test set accuracy

In [22]:
# TO DO: Print the results from the grid search (2 marks)
#Parameters
print("Best Parameters for Ridge Regression:", grid_ridge.best_params_)
print("Best Parameters for Random Forest:", grid_rf.best_params_)

#Train Score
print("Best Cross-Validation Score (Train) - Ridge Regression:", grid_ridge.best_score_)
print("Best Cross-Validation Score (Train) - Random Forest:", grid_rf.best_score_)

#Test Score
from sklearn.metrics import mean_squared_error, r2_score

y_pred_ridge = grid_ridge.best_estimator_.predict(X_test)
y_pred_rf = grid_rf.best_estimator_.predict(X_test)

ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_r2 = r2_score(y_test, y_pred_ridge)

rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)

print("Ridge Regression - Test MSE:", ridge_mse)
print("Ridge Regression - Test R²:", ridge_r2)

print("Random Forest - Test MSE:", rf_mse)
print("Random Forest - Test R²:", rf_r2)


Best Parameters for Ridge Regression: {'model__alpha': 0.01}
Best Parameters for Random Forest: {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 50}
Best Cross-Validation Score (Train) - Ridge Regression: -2.2677669126107336
Best Cross-Validation Score (Train) - Random Forest: -7.34376054685535
Ridge Regression - Test MSE: 2.892179738574027
Ridge Regression - Test R²: 0.6158300707331381
Random Forest - Test MSE: 8.090151597108344
Random Forest - Test R²: -0.0746195768426483


### Questions (8 marks)

1. Which models did you use scaling for? Why?
1. Which model and what parameters produced the best results?
1. Was this model a good fit? Why or why not?
1. Is there anything else we could do to try to improve model performance? Provide two ideas.

*ANSWER HERE*
1. The model we used for scaling was Ridge Regression. This is because Ridge Regression uses regularization and could change the way that features contribute if there is scaling in place. Random Forest does not need the scaling as it learns using a collection of decision trees which are not affected by features with varying magnitudes.

2. Ridge Regression with "model_alpha" parameter equal to 0.01 generated the best results. Our model resulted in a better cross validation score of -2.268, a MSE of 2.892 and R^2 score of 0.616. Comparing this to the Random Forest model, we found the cross validation to be -7.344, MSE of 8.090 and R^2 of -0.075 which is significantly worse and signified we should not be using neg_mean_square_error to get the best results from the Random Forest model.

3. Our model was an okay fit, but not the best. Although it was able to obtain decent scores from the training, the cross validation score is suggesting that the model could possibly be underfitting the data and is generalizing poorly. This is still more desirable than the Random Forest as it obtained a much larger MSE and performed worse on R^2 results as well.

4. The Ridge Regression model can be improved by removing highly correlated features from the dataset. This could improve our results as even though ridge regression removes certain features, it could be improved by selecting different features to see which ones are best removed. For the Random Forest, we could increase n_estimators to higher numbers like 500 to increase the stability of the model

### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE - BE SPECIFIC*
1. My code was sourced from similar sources in the previous assignments using scikit, geeksforgeeks and documentation for the functions and models we used in this assignment.

2. The steps were completed in order as they built upon each other and create a flow that makes sense for the training and results for the model.

3. Generative AI was used to explain concepts better such as "explain why a higher negative mean square error is better" as the results did not make sense at first since this is one of the first few times I use neg_mean_squre_error as a scoring metric. This improved my understanding and helped me understand the results obtained from our training and testing of the models

4. I faced the challenges of not choosing the correct categorical and numerical columns. This was due to the dataset being listed as some columns being categorical in the website we were given, but having numerical values such as 'open' or other columns of similar nature. After making some tweaks, using the stock as the only categorical column worked and numerical columns used appropriately even if listed as categorical on the website

## Part 2: Text Data (32 marks)

The purpose of this part of the assignment is to practice working with text data.

### 2.1: Load data (1 mark)
For this task, we will be using the hobbies dataset from the yellowbrick library. More information on the dataset can be found here: https://www.scikit-yb.org/en/latest/api/datasets/hobbies.html

In [51]:
# TO DO: Load the dataset (1 mark)
from yellowbrick.datasets import load_hobbies

corpus = load_hobbies()

### 2.2 Pre-processing (3 marks)

We will need to transform the data from strings to numeric. First, we will transform the data using `CountVectorizer(min_df=5)`.

In [52]:
# TO DO: Create CountVectorizer object (0.5 marks)
from sklearn.feature_extraction.text import CountVectorizer
text_data = corpus.data
y = corpus.target

vectorizer = CountVectorizer(min_df=5)

In [53]:
# TO DO: Fit vectorizer to data (0.5 marks)
X = vectorizer.fit(text_data)

In [54]:
# TO DO: What is the length of the vocabulary? (0.5 marks)
vocabulary_length = len(vectorizer.vocabulary_)

print("Length of the vocabulary:", vocabulary_length)

Length of the vocabulary: 3940


In [55]:
# TO DO: Transform the data (0.5 marks)
X = vectorizer.fit_transform(text_data)

In [56]:
# TO DO: What is the shape of the transformed data? (0.5 marks)
print("Shape of the transformed data:", X.shape)

Shape of the transformed data: (448, 3940)


In [57]:
# TO DO: Split data into training and testing sets (use random_state=0 and 10% of the data for testing) (0.5 marks)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

### 2.3: Grid Search (5 marks)

For the grid search, we want to compare the performance of Logistic Regression for different values of C. Initialize the parameter grid with parameter values that make sense for this model.

In [ ]:
# TO DO: Create parameter grid and initialize grid object (2 marks)
from sklearn.linear_model import LogisticRegression

y_train = np.array(y_train) # Needed to convert as it was having issues as a 'list' type

pipeline = Pipeline([
    ('classifier', LogisticRegression(solver='liblinear', max_iter=1000))  # Using liblinear because otherwise the sparse data was not compatible
])

param_grid = {
    'classifier__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')


In [61]:
# TO DO: Fit grid object to training data (1 mark)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('classifier',
                                        LogisticRegression(max_iter=1000,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                                           1000]},
             scoring='accuracy')

In [62]:
# TO DO: Print the results from the grid search (2 marks)
print("Best parameters found: ", grid_search.best_params_)

Best parameters found:  {'classifier__C': 1}


### 2.4: Additional Model Comparisons (9 marks)

### 2.4.1: Naive Bayes (3 marks)
We would like to compare the performance of Logistic Regression with one of the Naive Bayes models. Pick the Naive Bayes model that you think would best suit text data and implement below. Since we are not adjusting hyperparameters, we can use `cross_validate`.

In [63]:
# TO DO: Implement Naive Bayes model with cross-validate (2 marks)
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
pipeline_nb = Pipeline([
    ('classifier', MultinomialNB())  # Naive Bayes classifier
])

cv_results = cross_validate(pipeline_nb, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)

# TO DO: Print training and validation accuracies
print("Cross-validation results for Naive Bayes:")
print(f"Mean accuracy: {cv_results['test_score'].mean():.4f}")
print(f"Standard deviation of accuracy: {cv_results['test_score'].std():.4f}")

Cross-validation results for Naive Bayes:
Mean accuracy: 0.8809
Standard deviation of accuracy: 0.0256


In [65]:
# TO DO: Calculate and print test accuracy (1 mark)
pipeline_nb.fit(X_train, y_train)
test_accuracy = pipeline_nb.score(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

Test accuracy: 0.9556


### 2.4.2 Tf-idf (6 marks)

To try to improve the results, we can try using Tf-idf to tranform the text data based on the importance of each feature. We will need to use a pipeline and the original data for this section. Use `TfidfVectorizer(min_df=5)` and compare the results for both Logistic Regression and your selected Naive Bayes model. Use the Logistic Regression parameters from the previous section.

In [66]:
# TO DO: Split the data into training and testing sets (same values as previous section) (1 mark)
X_train, X_test, y_train, y_test = train_test_split(text_data, y, test_size=0.1, random_state=0)

In [ ]:
# TO DO: Implement Pipeline with Tf-idf vectorizer and both Logistic Regression and your selected Naive Bayes model (3 marks)
from sklearn.feature_extraction.text import TfidfVectorizer

#Pipeline with TfidfVectorizer and Logistic Regression
pipeline_lr_tfidf = Pipeline([
    ('vectorizer', TfidfVectorizer(min_df=5)),
    ('classifier', LogisticRegression(solver='liblinear', max_iter=1000))
])

#Pipeline with TfidfVectorizer and Naive Bayes
pipeline_nb_tfidf = Pipeline([
    ('vectorizer', TfidfVectorizer(min_df=5)),
    ('classifier', MultinomialNB())
])

#For Logistic Regression
param_grid_lr = {
    'classifier__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid_search_lr = GridSearchCV(pipeline_lr_tfidf, param_grid_lr, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_lr.fit(X_train, y_train)

#For Naive Bayes
param_grid_nb = {
    'classifier__alpha': [0.1, 0.5, 1, 2]
}

# Perform grid search for Naive Bayes
grid_search_nb = GridSearchCV(pipeline_nb_tfidf, param_grid_nb, cv=5, n_jobs=-1, scoring='accuracy')
grid_search_nb.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(min_df=5)),
                                       ('classifier', MultinomialNB())]),
             n_jobs=-1, param_grid={'classifier__alpha': [0.1, 0.5, 1, 2]},
             scoring='accuracy')

In [68]:
# TO DO: Print the results from the grid search (2 marks)
print("Logistic Regression - Best parameters found: ", grid_search_lr.best_params_)
print("Logistic Regression - Best cross-validation score: ", grid_search_lr.best_score_)

print("Naive Bayes - Best parameters found: ", grid_search_nb.best_params_)
print("Naive Bayes - Best cross-validation score: ", grid_search_nb.best_score_)

Logistic Regression - Best parameters found:  {'classifier__C': 100}
Logistic Regression - Best cross-validation score:  0.8932407407407407
Naive Bayes - Best parameters found:  {'classifier__alpha': 0.1}
Naive Bayes - Best cross-validation score:  0.8807716049382716


### Questions (10 marks)

1. Which Naive Bayes model did you pick? Why?
1. Which model and what parameters produced the best results?
1. Was this model a good fit? Why or why not?
1. Is there anything else we could do to try to improve model performance? Provide two ideas (must be different from Part 1).
1. Why did we need to implement a pipeline for Tf-idf and not CountVectorizer? What would happen if we didn't use one for Tf-idf?

*ANSWER HERE*
1. I chose Multinomial for the model. This choice was made because it works well with discrete features such as the text data we are working with in this dataset. Using the bag of words technique helps us with this kind of model and made the comparison very effective

2. The Logistic Regression model produced the best results. It produced a cross-validation score of 0.893 which is slightly higher than the NB model we used which produced a score of 0.881

3. Yes this model was a good fit. Our model received an acceptable cross validation score of 0.893, meaning it was not overfitting or underfitting. The classifier__C parameter with a value of 100 generated these results which was compared to 7 other values, giving us an ample set of parameter values that were tested and compared to find the best one. This is better than the NB model, but just slightly. This suggests that the NB model was also a good fit, but Logistic Regression was slightly better.

4. We could incorporate the ngram_range feature to create bigrams and trigrams to see if this improves the model in any way. Simultaneously, we could apply different alpha values that have a much larger range (ex. 0.01, 0.1, 1, 10, etc.) to see if the larger range of alpha values causes a significant change in model performance.

5. Without a pipeline we might fit the vectorizer on the training and test data, causing data leakage and potentially causing overfitting in our model which would not be present with a pipeline. It can also cause the data to be processed differently if there is no pipeline for Tf-idf, causing different results and changing the true accuracy of the model.

### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE - BE SPECIFIC*
1. The code was sourced from the same website as part 1, from scikit-learn, geeksforgeeks and their documentation for the used models and functions

2. The steps were also completed in order as they built upon each other and allows for a clear comparison and understanding of the process

3. I used prompts such as "Why is my pipeline not working" with the first grid search. This then explained that it was because CountVectorizer returned a sparse representation of the counts while LogisticRegression compared it using a dense one and needed to be modified to use the liblinear solver as that would make them compatible.

4. The main challenge I faced was finding out how to make my CountVectorizer and LogisticRegression comparison compatible as it took a very long time to figure out as there was not much clear documentation about how LogisticRegression is expecting a dense representation.

## Part 3: Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challangeing, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*
1. I really enjoyed learning about working with text data as I believe this is a very common type of data that would be used in real world situations and brings a proper application to the material we are learning. I did not enjoy not having the explanation of modifying our LogisticRegression model to make it compatible as it was not very clear and a slightly confusing process

2. I found the results very interesting using pipelines as it can help us compare different models quite efficiently by simplifying the preprocessing process for the models we are using before performing something such as a GridSearch